## Qs Construction

In [45]:
import kProcessor as kp
import random
import hashlib

In [119]:
# Mimic the function in kProcessor of coversion from str_to_int
def str_to_int(kmer):
    _map = {"A":0, "C":1, "T":2, "G":3}
    strint = 0
    for n in kmer:
        curr = _map[n]
        strint = strint | curr
        strint = strint << 2
        
    return strint >> 2

def int_to_str(kmer, kSize):
    _map = {0: 'A', 1: 'C', 2: 'T', 3: 'G'}
    kmer_str = ""
    for i in range(kSize, 0, -1):
        base = (kmer >> (i*2-2)) & 3
        ch = _map[base]
        kmer_str += ch
    
    return kmer_str

# Function to match 2 kmers with common first Q*2 bits or First Q Chars
# Return bool value, True if match, False if not
def matchingKmers(kmer1_int, kmer2_int, kSize, Q):
    msk = (~(-1<<Q*2)) << (kSize*2 - Q*2)
    return ((kmer1_int ^ kmer2_int) & msk == 0)

def mask(kSize, Q):
    return (~(-1<<Q*2)) << (kSize*2 - Q*2)

# Take list of colors, sort it, and create a return a hash value
def create_super_color(colors):
    val = hashlib.md5(str(sorted(colors)).encode()).hexdigest()[:9]
    print("Hashing: %s to %s" % (str(sorted(colors)) , val) )
    return val

## Index Loading

In [61]:
kList_str = {}
kList = {}
## Double loading will be changed in the future after implementing coloredkDataFrame iterator
kf = kp.kDataFrame.load("idx_min_test")
ckf = kp.colored_kDataFrame.load("idx_min_test")
kSize = kf.getkSize()

# Determine minQ and maxQ and get list of masks & superColorsDIct initialization

In [120]:
minQ = 9
stepQ = 1
maxQ = 15 #kSize
superColors = {}
temp_superColors = {}
superColorsCount = {}
masks = {}

for Q in range(maxQ, minQ-1, -stepQ):
    masks[Q] = mask(kSize, Q)
    superColors[Q] = {}
    superColorsCount[Q] = {}
    temp_superColors[Q] = []

print(temp_superColors)
print(superColors)
print(superColorsCount)

list(map(bin,masks.values()))

{15: [], 14: [], 13: [], 12: [], 11: [], 10: [], 9: []}
{15: {}, 14: {}, 13: {}, 12: {}, 11: {}, 10: {}, 9: {}}
{15: {}, 14: {}, 13: {}, 12: {}, 11: {}, 10: {}, 9: {}}


['0b111111111111111111111111111111000000000000',
 '0b111111111111111111111111111100000000000000',
 '0b111111111111111111111111110000000000000000',
 '0b111111111111111111111111000000000000000000',
 '0b111111111111111111111100000000000000000000',
 '0b111111111111111111110000000000000000000000',
 '0b111111111111111111000000000000000000000000']

In [125]:
it = kf.begin()
prev_kmer = it.getHashedKmer()
prev_kmer_color = it.getKmerCount()

count = 5
_samples_ids_cache = kp.colorsList()
# Create list of kmers
while it != kf.end():
    
    if count == 0:
        break
    count -= 1
    
    it.next()
    curr_kmer = it.getHashedKmer()
    curr_kmer_color = it.getKmerCount()
    
    # Apply XOR to kmer1 and kmer2 (single time per iteration)
    xor = prev_kmer ^ curr_kmer
    
    # Apply all masks with all Qs
    for Q, MASK in masks.items():
        matched = not bool(xor & MASK) # True if there's match, False if not
        if matched:
            print("Matching Q%d %s & %s | TRUE" % (Q, int_to_str(prev_kmer, kSize),int_to_str(curr_kmer, kSize)))
            temp_superColors[Q] += [prev_kmer_color, curr_kmer_color]
        else:
            print("Matching Q%d %s & %s | FALSE" % (Q, int_to_str(prev_kmer, kSize),int_to_str(curr_kmer, kSize)))
            # Get all Samples of these co`lors
            samples_ids = []
            for _col in temp_superColors[Q]:
                ckf.getSamplesIDForColor(_col, _samples_ids_cache)
                samples_ids += _samples_ids_cache
                super_color = create_super_color(temp_superColors[Q])
                
                # Check if the superColor already exist
                # If yes: increment the count to one
                # If No:  Insert the new superColor and set the count to 1
                if super_color not in superColors[Q]:
                    superColors[Q][super_color] = set(samples_ids)
                    superColorsCount[Q][super_color] = 1
                    
                else:
                    superColorsCount[Q][super_color] += 1
                    
                    
    
#     for Q in range(maxQ, minQ, -stepQ):
#         if(matchingKmers(curr_kmer, prev_kmer, kf.getkSize(), Q)):
#             print("Matching Q%d %s & %s | TRUE" % (Q, int_to_str(prev_kmer, kSize),int_to_str(curr_kmer, kSize)))
#         else:
#             print("Matching Q%d %s & %s | FALSE" % (Q, int_to_str(prev_kmer, kSize),int_to_str(curr_kmer, kSize)))
    print("---------------------------------------")
    
    prev_kmer = curr_kmer

Matching Q15 AAAAAAAAAAACCTTAACAAA & AAAAAAAAAACCTTAACAAAT | FALSE
Matching Q14 AAAAAAAAAAACCTTAACAAA & AAAAAAAAAACCTTAACAAAT | FALSE
Matching Q13 AAAAAAAAAAACCTTAACAAA & AAAAAAAAAACCTTAACAAAT | FALSE
Matching Q12 AAAAAAAAAAACCTTAACAAA & AAAAAAAAAACCTTAACAAAT | FALSE
Matching Q11 AAAAAAAAAAACCTTAACAAA & AAAAAAAAAACCTTAACAAAT | FALSE
Matching Q10 AAAAAAAAAAACCTTAACAAA & AAAAAAAAAACCTTAACAAAT | TRUE
Matching Q9 AAAAAAAAAAACCTTAACAAA & AAAAAAAAAACCTTAACAAAT | TRUE
---------------------------------------
Matching Q15 AAAAAAAAAACCTTAACAAAT & AAAAAAAAACCTTAACAAATG | FALSE
Matching Q14 AAAAAAAAAACCTTAACAAAT & AAAAAAAAACCTTAACAAATG | FALSE
Matching Q13 AAAAAAAAAACCTTAACAAAT & AAAAAAAAACCTTAACAAATG | FALSE
Matching Q12 AAAAAAAAAACCTTAACAAAT & AAAAAAAAACCTTAACAAATG | FALSE
Matching Q11 AAAAAAAAAACCTTAACAAAT & AAAAAAAAACCTTAACAAATG | FALSE
Matching Q10 AAAAAAAAAACCTTAACAAAT & AAAAAAAAACCTTAACAAATG | FALSE
Hashing: [1, 1, 1, 1, 1, 1] to 45e657145
Hashing: [1, 1, 1, 1, 1, 1] to 45e657145
Hashing: [

In [118]:
superColors

{15: {},
 14: {},
 13: {},
 12: {},
 11: {},
 10: {'e5df5f641': {1}, 'bfb10fa9e': {1}},
 9: {'45e657145': {1}, '7107cc562': {1}}}

In [110]:
superColorsCount

{15: {},
 14: {},
 13: {},
 12: {},
 11: {},
 10: {'e5df5f641': 8, 'bfb10fa9e': 16, '45e657145': 24, '134c522e4': 32},
 9: {'45e657145': 12, '7107cc562': 24, 'f2e5a57fd': 36, '1d50487b6': 48}}

In [101]:
temp_superColors

{15: [], 14: [], 13: [], 12: [], 11: [], 10: [1, 1], 9: [1, 1, 1, 1, 1, 1]}

In [59]:
ckf.getSamplesIDForColor?

Signature: ckf.getSamplesIDForColor(color:'uint32_t', result:'colorsList') -> 'void'
Docstring: <no docstring>
File:      ~/miniconda3/envs/kcluster/lib/python3.6/site-packages/kProcessor.py
Type:      method


In [ ]:
for Q in range(maxQ, minQ, -stepQ):
    print(Q)

In [52]:
not bool(0)

True